In [28]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets


In [29]:
model = models.resnet50(pretrained=True)

In [30]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [32]:
print(device)
print(model)

cuda
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride

In [33]:
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [34]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # Grayscale normalization
])

In [35]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [36]:
train_dataset = ImageFolder(root='/kaggle/input/chest-xrayfinal/chest_xray/train', transform=train_transform)
test_dataset = ImageFolder(root='/kaggle/input/chest-xrayfinal/chest_xray/test', transform=test_transform)

In [37]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [38]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

In [40]:
num_epochs = 45  # Change based on your requirement

for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    model.train()
    running_loss = 0.0
    i=0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        i+=1
        print("-",end="")
        if (i % 100 == 0):
          print(i)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print("Validating",end=" ")
        # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs, labels = inputs.to(device), labels.to(device)  # <--
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Metrics and early stopping
    train_loss = running_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    val_acc = correct / total

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

# Save the fine-tuned model
torch.save(model.state_dict(), "finetuned_resnet.pth")


Epoch [1/45]
----------------------------------------------------------------------------------Validating Train Loss: 0.0079 | Val Loss: 2.1369 | Val Acc: 0.4119
Epoch [2/45]
----------------------------------------------------------------------------------Validating Train Loss: 0.0066 | Val Loss: 1.8174 | Val Acc: 0.5513
Epoch [3/45]
----------------------------------------------------------------------------------Validating Train Loss: 0.0064 | Val Loss: 3.2789 | Val Acc: 0.5128
Epoch [4/45]
----------------------------------------------------------------------------------Validating Train Loss: 0.0059 | Val Loss: 2.1256 | Val Acc: 0.5689
Epoch [5/45]
----------------------------------------------------------------------------------Validating Train Loss: 0.0055 | Val Loss: 3.6089 | Val Acc: 0.3926
Epoch [6/45]
----------------------------------------------------------------------------------Validating Train Loss: 0.0052 | Val Loss: 3.6411 | Val Acc: 0.4327
Epoch [7/45]
---------------

KeyboardInterrupt: 

In [27]:
model_save_path = "/kaggle/working/pneumonia_model_10.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved at {model_save_path}")

Model saved at /kaggle/working/pneumonia_model.pth


In [ ]:
num_epochs = 10  # Change based on your requirement

for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    model.train()
    running_loss = 0.0
    i=0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        i+=1
        print("-",end="")
        if (i % 100 == 0):
          print(i)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print("Validating",end=" ")
        # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs, labels = inputs.to(device), labels.to(device)  # <--
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Metrics and early stopping
    train_loss = running_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    val_acc = correct / total

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

# Save the fine-tuned model
torch.save(model.state_dict(), "finetuned_resnet.pth")
